##Prepare and split the data

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataFilePath = '/content/drive/MyDrive/ALL_eminem.txt'
with open(dataFilePath, 'r') as file:
  data = (file.read())

In [ ]:
import re

#Remove the artist name of the top of the verses
data = re.sub('\[.*?\]\n', "", data) 

# split into verses
data_songs = data.split('\n\n')

In [ ]:
from math import floor
train_val_loc = floor(len(data_songs)*0.7)
val_test_loc = floor(len(data_songs)*0.9)
train_data = data_songs[:train_val_loc]
val_data = data_songs[train_val_loc:val_test_loc]
test_data = data_songs[val_test_loc:]

print("train_verses =", len(train_data))
print("val_verses =", len(val_data))
print("test_verses =", len(test_data))
print("total verses =", len(data_songs))

train_verses = 1203
val_verses = 344
test_verses = 172
total verses = 1719


In [ ]:
def to_gpt2_in_text(verse_list: list, path: str):
  data = ""
  for verse in verse_list:
    verse = verse.strip()
    verse = re.sub("\s", " ", verse) # replace all types of spaces like tabs and so on with a normal space 
    bos_token = ''
    eos_token = ''
    data += bos_token + ' ' + verse + ' ' + eos_token + '\n'
  file = open(path, 'w')
  file.write(data)
  
to_gpt2_in_text(train_data, "train_data.txt")
to_gpt2_in_text(val_data, "val_data.txt")
to_gpt2_in_text(test_data, "test_data.txt")

## Installation of necessary packages

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.0 MB/s eta 0:00:00


In [ ]:
! pip install git+https://github.com/huggingface/transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-vuqlu7cn
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-vuqlu7cn
  Resolved https://github.com/huggingface/transformers to commit 4e1522d65ac5ff5a4a90dc2a223fc20078a85744
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.29.0.dev0-py3-none-any.whl size=6990209 sha256=a262fc36d9da216377c9d0fbc3f82e6d8c4fe0b6193df4c783ecb0dcc1c60f51
  Stored in directory: /tmp/pip-ephem-wheel-cache-sermne3i/wheels/14/a0/7b/8f6b25ba4110aa215fcb8d6aedd6cd4f9b9b6619190999ac2b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.28.1
    Uninstalling transformers-

In [ ]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 18.4 MB/s eta 0:00:00


In [ ]:
! pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 6.5 MB/s eta 0:00:00


## Fine tuning of model

In [ ]:
!python '/content/drive/MyDrive/run_clm.py' \
  --output_dir='/content/drive/MyDrive/emNN_finetuned'\
  --model_type=gpt2 \
  --model_name_or_path='gpt2' \
  --do_train \
  --train_file='train_data.txt'\
  --do_eval \
  --validation_file='val_data.txt'\
  --per_device_train_batch_size=2 \
  --per_device_eval_batch_size=2 \
  --learning_rate 5e-5 \
  --num_train_epochs=5

Exception ignored in: <function _xla_gc_callback at 0x7f493cfc74c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 
2023-04-26 11:17:31.105474: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
04/26/2023 11:17:34 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False
Extracting data files: 100% 2/2 [00:00<00:00, 1836.79it/s]
Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-9518d178f7da0953/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.
100% 2/2 [00:00<00:00, 766.92it/s]
Running tokenizer on dataset:   0% 0/1203 [00:00<?, ? examples/s][WARNING|tokenization_utils_base.py:3581] 2023-04-26 11:17:38,049 >> Token indices sequence length is longe

## Compare gpt2 and the finetuned gpt2 on the unseen test set

In [ ]:
!python /content/drive/MyDrive/run_clm.py --output_dir='/content/drive/MyDrive/emNN_finetuned_eval' --model_type='gpt2' --model_name_or_path='/content/drive/MyDrive/emNN_finetuned' --do_eval --validation_file='test_data.txt' --per_device_eval_batch_size=2

2023-04-26 10:54:46.186071: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
04/26/2023 10:54:50 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False
Extracting data files: 100% 1/1 [00:00<00:00, 820.64it/s]
Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-30b327259ed38b6f/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.
100% 1/1 [00:00<00:00, 806.60it/s]
100% 14/14 [00:02<00:00,  5.33it/s]
***** eval metrics *****
  eval_loss               =     3.8621
  eval_runtime            = 0:00:03.25
  eval_samples            =         27
  eval_samples_per_second =      8.303
  eval_steps_per_second   =      4.305
  perplexity              =    47.5646


In [ ]:
!python run_clm.py --output_dir='/content/drive/MyDrive/eminem_gpt2_eval' --model_type='gpt2' --model_name_or_path='gpt2' --do_eval --validation_file='test_data.txt' --per_device_eval_batch_size=2

2023-04-25 19:34:20.292802: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
04/25/2023 19:34:24 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
Extracting data files: 100% 1/1 [00:00<00:00, 1007.28it/s]
Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-24fe4eec150f3ab6/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.
100% 1/1 [00:00<00:00, 851.64it/s]
100% 14/14 [00:02<00:00,  5.23it/s]
***** eval metrics *****
  eval_loss               =               121.334
  eval_runtime            =            0:00:03.28
  eval_samples            =                    27
  eval_samples_per_second =                 8.225
  eval_steps_per_second   =                 4.265
  perplexity              = 4.951115699746246e+52


Massive improvement in perplexity after fine tuning



## Generate from prompt

In [ ]:
!python '/content/drive/MyDrive/run_generation.py' \
  --model_type gpt2 \
  --model_name_or_path '/content/drive/MyDrive/emNN_finetuned' \
  --prompt "computer science is love computer science is life" \
  --stop_token "" \
  --k 50 \
  --length=500 \
  --num_return_sequences 5

2023-04-26 11:29:57.245767: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
04/26/2023 11:29:58 - WARNING - __main__ - device: cuda, n_gpu: 1, 16-bits training: False
04/26/2023 11:30:05 - INFO - __main__ - Namespace(model_type='gpt2', model_name_or_path='/content/drive/MyDrive/emNN_finetuned', prompt='computer science is love computer science is life', length=500, stop_token='', temperature=1.0, repetition_penalty=1.0, k=50, p=0.9, prefix='', padding_text='', xlm_language='', seed=42, no_cuda=False, num_return_sequences=5, fp16=False, device=device(type='cuda'), n_gpu=1)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
computer science is love computer science is life Computer science is sex appeal computer sci

## Wrapping it into a function

In [ ]:
import subprocess

def generate_lyric(start: str, length: int, topk: int) -> list[str]:
  length_flag = "--length=" + str(length)
  result = subprocess.run(['python', '/content/drive/MyDrive/run_generation.py', '--model_type', 'gpt2', '--model_name_or_path', '/content/drive/MyDrive/emNN_finetuned', '--prompt', start, '--stop_token', '', '--k', '50', length_flag, '--num_return_sequences', str(topk)], capture_output=True)
  result_str = result.stdout.decode('utf-8')
  result_str = re.sub('=== GENERATED SEQUENCE 1 ===\n', "", result_str)
  result_str = re.sub('<EOS>', '', result_str)
  result_str = re.sub('<BOS>', '', result_str)
  result_list = re.split('=== GENERATED SEQUENCE \d ===\n', result_str)
  return result_list


In [ ]:
lyrics_res = generate_lyric("Man don't do this", 200, 10)

In [ ]:
print(lyrics_res[0].replace('"', '\n'))

Man don't do this for you no more

 
Stop making me look like an idiot.

 
I'm on my own when it comes to my kids. My brain is always full of crap

 
What the fuck? I'm acting different than a teenager

 
It was my mother's fault. She tried to stop us when we were kids and we grew up with the opposite traits

 
I'm from the block and she said 'Look at my kid, he's not behaving as a normal kid.'

 
You wanna think that I'm retarded?
 
No.

 
Shut up, you can't get your butt kicked!
 
I'm not retarded! I'm just mentally retarded!

 
I'm a kid with no sense of humor and no taste in things

 
That's why I never give a fuck.
 
It ain't my fault that you've been treated worse than me

 
Look at these pictures of my brain! What is it you think I

